# Transformaciones

In [ ]:
import json
import openpyxl
import pandas as pd
import re
import csv 
from datetime import datetime


### 1 - Dataset "Homicidios - Hechos"
    Información sobre homicidios en siniestros viales ocurridos en la Ciudad desde 
    el año 2016 hasta el 2021. Los datos incluyen fecha y ubicación del hecho y 
    tipo de transporte involucrado.

In [54]:
#Cargamos el Dataset proporcionado en formato excel
df = pd.read_csv('./Datasets/homicidios - HECHOS.csv')

# Mostrar el DataFrame
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     696 non-null    object 
 1   N_VICTIMAS             696 non-null    int64  
 2   FECHA                  696 non-null    object 
 3   AAAA                   696 non-null    int64  
 4   MM                     696 non-null    int64  
 5   DD                     696 non-null    int64  
 6   HORA                   696 non-null    object 
 7   HH                     696 non-null    object 
 8   LUGAR_DEL_HECHO        696 non-null    object 
 9   TIPO_DE_CALLE          696 non-null    object 
 10  Calle                  695 non-null    object 
 11  Altura                 129 non-null    float64
 12  Cruce                  525 non-null    object 
 13  Dirección Normalizada  688 non-null    object 
 14  COMUNA                 696 non-null    int64  
 15  XY (CA

In [55]:
# Convertimos 'FECHA' a datetime
df['FECHA'] = pd.to_datetime(df['FECHA'])

# Filtramos valores anómalos en 'HORA'
invalid_times = df[~df['HORA'].str.match(r'^\d{1,2}:\d{2}:\d{2}$')]
#print("Valores anómalos en 'HORA':")
#print(invalid_times)

# Reemplazamos valores anómalos con un valor por defecto '00:00:00'
df['HORA'] = df['HORA'].apply(lambda x: '00:00:00' if not re.match(r'^\d{1,2}:\d{2}:\d{2}$', x) else x)

# Convertimos 'HORA' a time
df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M:%S').dt.time

#Presenta un solo valor anómalo en el id 2019-0103

In [56]:
# Creamos la columna DIA_SEMANA para análisis temporal
df['DIA_SEMANA'] = df['FECHA'].dt.day_name()

In [57]:
# Rellenamos valores nulos en 'Calle' y 'Cruce' con 'unknown'
df['Calle'].fillna('unknown', inplace=True)
df['Cruce'].fillna('unknown', inplace=True)

In [58]:
# Eliminar la columna 'Altura'
df = df.drop(columns=['Altura', 'Calle', 'Cruce'])

In [59]:
# Convertimos las coordenadas a numérico si están en formato de cadena
df['pos x'] = pd.to_numeric(df['pos x'], errors='coerce')
df['pos y'] = pd.to_numeric(df['pos y'], errors='coerce')

# Verificamos si hay valores nulos después de la conversión y tratarlos si es necesario
#print("Valores nulos a eliminar en 'pos x':", df['pos x'].isnull().sum())
#print("Valores nulos a eliminar en 'pos y':", df['pos y'].isnull().sum())

# Eliminamos filas con valores nulos en 'pos x' y 'pos y'
df = df.dropna(subset=['pos x', 'pos y'])


In [60]:
# Verificamos y convertimos las columnas categóricas
categorical_columns = ['TIPO_DE_CALLE', 'COMUNA', 'PARTICIPANTES', 'VICTIMA', 'ACUSADO']
for column in categorical_columns:
    df[column] = df[column].astype('category')

# Comprobamos de los cambios
print(df.dtypes)
print(df.head())

ID                               object
N_VICTIMAS                        int64
FECHA                    datetime64[ns]
AAAA                              int64
MM                                int64
DD                                int64
HORA                             object
HH                               object
LUGAR_DEL_HECHO                  object
TIPO_DE_CALLE                  category
Dirección Normalizada            object
COMUNA                         category
XY (CABA)                        object
pos x                           float64
pos y                           float64
PARTICIPANTES                  category
VICTIMA                        category
ACUSADO                        category
DIA_SEMANA                       object
dtype: object
          ID  N_VICTIMAS      FECHA  AAAA  MM  DD      HORA HH  \
0  2016-0001           1 2016-01-01  2016   1   1  04:00:00  4   
1  2016-0002           1 2016-01-02  2016   1   2  01:15:00  1   
2  2016-0003           1 201

In [61]:
# Guardar el DataFrame procesado en un archivo parquet
df.to_parquet('./Datasets/homicidios-HECHOS-limpio.parquet', index=False)